In [ ]:
from fastai.vision.all import *

In [ ]:
datapath = Path("/kaggle/input/rsna-str-pulmonary-embolism-detection/")
train_df = pd.read_csv(datapath/'train.csv')
test_df = pd.read_csv(datapath/'test.csv')
imagepath = Path("/kaggle/input/rsna-str-pe-detection-jpeg-256/")

In [ ]:
train_df.head(3)

In [ ]:
train_qi = train_df.groupby(['StudyInstanceUID'])['pe_present_on_image'].agg('mean')

In [ ]:
train_df.pe_present_on_image.mean()

In [ ]:
labels_dict = dict(zip(train_df['SOPInstanceUID'], train_df['pe_present_on_image']))

In [ ]:
len(labels_dict)

In [ ]:
unique_pids = train_df.StudyInstanceUID.unique()

In [ ]:
n = len(unique_pids)
nvalid = int(n*0.1); nvalid, n

In [ ]:
valid_pids = np.random.permutation(unique_pids)[:nvalid]
len(valid_pids)

In [ ]:
files = get_image_files(imagepath)

In [ ]:
len(files)

In [ ]:
# sample_files = np.random.choice(files, 100000)
# len(sample_files)

In [ ]:
def get_dls(files, size=256, bs=128):
    trn_idxs, val_idxs = [],[]
    for i,o in enumerate(files):
        if o.parent.parent.name in valid_pids: val_idxs.append(i)
        else: trn_idxs.append(i)
        
    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(size, min_scale=0.9)], 
            [lambda o: labels_dict[o.stem.split("_")[1]], Categorize()]]

    dsets = Datasets(files, tfms=tfms, splits=(trn_idxs, val_idxs))

    batch_tfms = [IntToFloatTensor]
    dls = dsets.dataloaders(bs=bs, after_batch=batch_tfms)
    return dls

In [ ]:
dls = get_dls(files, bs=256)

In [ ]:
dls.show_batch()

In [ ]:
sqrmom=0.99
mom=0.95
beta=0.
eps=1e-4
opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

In [ ]:
learn = cnn_learner(dls, xresnet34, opt_func=opt_func, pretrained=True)
learn.to_fp16();

In [ ]:
learn.lr_find()

In [ ]:
learn.path = Path(".")

In [ ]:
learn.fine_tune(2, 3e-1)

In [ ]:
learn.save('basemodel-ft')

### Simulate Score

In [ ]:
# res = learn.get_preds(with_loss=True)

# preds, targs, losses = res

# valid_qi = tensor([train_qi[o.parent.parent.name] for o in dls.valid_ds.items])

# w = 0.07361963
# (sum(w*valid_qi*losses) + (len(valid_pids)*0.69))/(sum(w*valid_qi)+ len(valid_qi))

# np.sum([0.0736196319,
# 0.09202453988,
# 0.1042944785,
# 0.1042944785,
# 0.1877300613,
# 0.06257668712,
# 0.06257668712,
# 0.2346625767,
# 0.0782208589])

